# FFN 以及改进激活函数

FFN（FeedForwardNet）前馈神经网络一般是认为用于存储知识，标准的FFN一般由升维部分和降维部分组成。  
$$ \mathrm{FFN} (x)= RELU(xW_1 + b_1)W_2 +b2$$
其中的x的shape为 (batch_size, seq_len, hidden_dim), W_1的shape（h, 4h）,W_2的shape（4h, h）

可以从看到$W_1$体现了升维的过程，$W_2$则是降维过程。激活函数体现了输入输出的复杂线性关系，可以替换为其他的激活函数，例如现在常用的GELU（Gaussian Error Linear Unit，GELU）以及swichGELU。

In [1]:
import math
import torch
from torch import nn
import torch.nn.functional as F

class Config:
    def __init__(self, n_embd=512, dropout=0.1):
        self.n_embd = n_embd  # 嵌入维度
        self.dropout = dropout  # Dropout比率


# FFN实现--(实际上就是一个MLP)
class FeedForwardNet(nn.Module):
        def __init__(self, config):
            super().__init__()
            self.net = nn.Sequential(
                  nn.Linear(config.n_embd, 4 * config.n_embd),
                  nn.ReLU(),
                  nn.Linear(4 * config.n_embd, config.n_embd),
                  nn.Dropout(config.dropout)
            )

        def forward(self, x):
            return self.net(x)    

In [2]:
# 测试
config = Config(n_embd=64, dropout=0.2)
ffn = FeedForwardNet(config)

x = torch.randn(3, 8, config.n_embd)
with torch.no_grad():
    y = ffn(x)

print("输入 shape :", x.shape)
print("输出 shape :", y.shape)
print("参数量   :", sum(p.numel() for p in ffn.parameters())) 
# (64,)-->(64,64*4)+bias(64*4,) 16640
# (64,64*4)-->(64,)+bias(64,) 16448

输入 shape : torch.Size([3, 8, 64])
输出 shape : torch.Size([3, 8, 64])
参数量   : 33088


## ReLU的改进

### 1.ReLU
ReLU的公式很简单：
$$ ReLU (x)= \max{(0, x)}$$

### 2.GELU 
从GPT以及BERT以来GRELU使用越来越多：  
$$ GELU (x) = xP(X \le x) = x \Phi(x) $$
其中$\Phi(x)$是标准正态分布的累计函数：
$$\Phi(x) = \frac{1}{2}(1 + erf(\frac{x}{\sqrt{x}}))$$
并且这里的erf是误差函数：
$$erf(x) = \frac{2}{\sqrt{\pi}}\int_{0}^{x}{e^{-t^2}dt}$$


### 3.SwiGLU
SwiGLU（或称swishGLU）是swish激活函数和GLU门控单元的结合。  
FFN层把偏置量写入权重里就变成：
$$FFN(x) = \mathrm{ActiveFuntion}(xW_1)W_2$$
* swish激活函数：
$$\mathrm{Swish}(x) = x\sigma(\beta x) $$
其中的$\beta$是一个超参数，当$\beta = 1$时就变成SiLU()。  
因此采用swish激活函数的FFN:
$$FFN (W_1, W_2, x) = \mathrm{Swish}(xW_1)W_2$$
其中的超参数为$W_1, W_2$
* GLU门控单元
GLU，Gated Linear Units是一种门控结构（有参数，因此相对于普通的激活函数多了一个 gate 矩阵），通过 sigmoid 控制不同维度的激活。公式如下：
$$GLU(W, x, V, b, c) = (Wx + b) \otimes sigmoid(Vx + c)$$
而我们都知道 FFN 是一个升高维度，然后降低维度的过程，因此可以写成，W2 是一个降低维度的参数，W1 是升高维度的过程，而 W3 是一个 Gate 需要用到的参数矩阵。
$$FFN (w_{up},w_{down},w_{gate})= w_{down} \times (w_{uo}x \otimes \mathrm{Swish}(w_{gate}x))$$

假设输入的 hidden_dim 大小是 hidden_dim，那么中间层（up 后的维度）大小是 mid_dim， 具体计算逻辑如下：

mid_dim = int(8 * hidden_dim / 3)  
#multiple_of：make SwiGLU hidden layer size multiple of large power of 2  
mid_dim = multiple_of * ((mid_dim + multiple_of - 1) // multiple_of)  

#multiple_of 一般设置为 256， LLaMA 和 GPT等模型用来优化计算效率  

# 3.带有swishGLU的FFN实现

In [3]:
class FFNExper(nn.Module):
    def __init__(self, hidden_dim, dropout):
        super().__init__()

        hidden_dim = hidden_dim
        mid_dim = hidden_dim * 8 // 3

        self.up = nn.Linear(hidden_dim, mid_dim, bias=False)
        self.down = nn.Linear(mid_dim, hidden_dim, bias=False)
        self.gate = nn.Linear(hidden_dim, mid_dim, bias=False)

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        output = self.dropout(
            self.down(
                F.silu(self.gate(x))* self.up(x)
            )
        )
        return output

In [4]:
# 测试
torch.manual_seed(42)
batch, seq_len, hidden = 2, 7, 512
x = torch.randn(batch, seq_len, hidden)

# 实例化模型
ffn = FFNExper(512, dropout=0.1)
ffn.eval()

# 前向一次
with torch.no_grad():
    y = ffn(x)

print("in_shape", x.shape)
print("our_shpe", y.shape)

in_shape torch.Size([2, 7, 512])
our_shpe torch.Size([2, 7, 512])


In [5]:
mid_dim =  hidden* 8 // 3
gate_out = F.silu(ffn.gate(x))  # [B, T, mid_dim]
up_out   = ffn.up(x)            # [B, T, mid_dim]
print("gate(x) 前10个值:", "\n", gate_out[0, 0, :10])
print("up(x)   前10个值:", "\n", up_out[0, 0, :10])
print("silu(gate) * up 前10个值:", "\n", (gate_out * up_out)[0, 0, :10])

gate(x) 前10个值: 
 tensor([-0.1939, -0.1362,  0.4813, -0.0365,  0.4199,  0.2744, -0.1579, -0.0501,
        -0.2211,  0.7552], grad_fn=<SliceBackward0>)
up(x)   前10个值: 
 tensor([-7.3304e-01, -9.3535e-04,  1.2379e+00, -4.0043e-01, -6.3098e-01,
         1.8584e-01,  6.9051e-01,  9.9118e-01,  4.9408e-01,  1.3586e+00],
       grad_fn=<SliceBackward0>)
silu(gate) * up 前10个值: 
 tensor([ 1.4213e-01,  1.2744e-04,  5.9577e-01,  1.4631e-02, -2.6498e-01,
         5.1000e-02, -1.0906e-01, -4.9707e-02, -1.0923e-01,  1.0260e+00],
       grad_fn=<SliceBackward0>)
